<a href="https://colab.research.google.com/github/anagraw/Detect_AI_Generated_Text/blob/main/Detect_Fake_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -q keras-core --upgrade
!pip install keras-nlp tensorflow==2.11 tensorflow-text==2.11 --upgrade -q
!pip install --upgrade -q wandb git+https://github.com/soumik12345/wandb-addons

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


#How to Deal with errors importing keras_nlp?
1. !pip uninstall keras keras-core keras-nlp
2. !pip install keras==2.15.0 keras-core==0.1.7 keras-nlp==0.6.3
3. Restart the session on Colab

In [1]:
import os
#os.environ["KERAS_BACKEND"] = "jax"  # "jax" or "tensorflow" or "torch"
#os.environ["WANDB_SILENT"] = "false" # for wandb

import keras_nlp
import keras_core as keras
import keras_core.backend as K


import torch
# import jax
import tensorflow as tf
# from tensorflow import keras
# import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

Using TensorFlow backend


<ipython-input-1-dd92fcfa4220>:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


In [25]:
!pip uninstall keras keras-core keras-nlp


Found existing installation: keras 2.11.0
Uninstalling keras-2.11.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras-2.11.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras/*
Proceed (Y/n)? Y
  Successfully uninstalled keras-2.11.0
Found existing installation: keras-core 0.1.7
Uninstalling keras-core-0.1.7:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras_core-0.1.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras_core/*
Proceed (Y/n)? Y
  Successfully uninstalled keras-core-0.1.7
Found existing installation: keras-nlp 0.6.4
Uninstalling keras-nlp-0.6.4:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras_nlp-0.6.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras_nlp/*
Proceed (Y/n)? Y
  Successfully uninstalled keras-nlp-0.6.4


In [27]:
!pip install keras==2.15.0 keras-core==0.1.7 keras-nlp==0.6.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00
  Using cached keras_core-0.1.7-py3-none-any.whl (950 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 11.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.7 MB/s eta 0:00:00
  Using cached tensorflow_text-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
print("TensorFlow:", tf.__version__)
# print("JAX:", jax.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)


TensorFlow: 2.15.0
Keras: 0.1.7
KerasNLP: 0.6.3


CONFIGURATION

In [19]:
class CFG:
    verbose = 0  # Verbosity

    wandb = True  # Weights & Biases logging
    competition = 'llm-detect-ai-generated-text'  # Competition name
    _wandb_kernel = 'awsaf49'  # WandB kernel
    comment = 'DebertaV3-MaxSeq_200-ext_s-torch'  # Comment description

    preset = "deberta_v3_base_en"  # Name of pretrained models
    sequence_length = 200  # Input sequence length

    device = 'TPU'  # Device

    seed = 42  # Random seed

    num_folds = 5  # Total folds
    selected_folds = [0, 1, 2]  # Folds to train on

    epochs = 3 # Training epochs
    batch_size = 3  # Batch size
    drop_remainder = True  # Drop incomplete batches
    cache = True # Caches data after one iteration, use only with `TPU` to avoid OOM

    scheduler = 'cosine'  # Learning rate scheduler

    class_names = ["real", "fake"]  # Class names [A, B, C, D, E]
    num_classes = len(class_names)  # Number of classes
    class_labels = list(range(num_classes))  # Class labels [0, 1, 2, 3, 4]
    label2name = dict(zip(class_labels, class_names))  # Label to class name mapping
    name2label = {v: k for k, v in label2name.items()}  # Class name to label mapping

In [20]:
keras.utils.set_random_seed(CFG.seed)
